## OpenStreetMap Data Fetching
**This code queries OpenStreetMap API (https://overpass-turbo.eu) for toll facilities, and their attributes any part of the world. After API call, data is mined and munged to get to a form that can be added to IBTTA's database**

- The data being queried and pulled here is all the tolling facilities in countries all around the world, including facility names, geolocations, relation with adjacent ways and nodes
- In addition to the above, the code indirectly calculates the length of all ways (Center Lane Miles of all Toll Roads) through obtaining the coordinates of each node on the way.
- Distances between individual nodes are calculated (with WSG84 projection) through GeoPy package and then rolled up to get the length of the entire toll road. All of these metrics are vital for the tolling industry.
- This codes starts with California, and is optimzed to take any country/state as an input with minimal manual work.

The complexity of OSM geodata comes from how it's defined in the platform and input by users. For example, in reality what is conceptually one Highway with one name, one length, etc., in OSM is split into many "ways" (smaller connected segments), and "nodes" (points forming each segment), with each segment having its own set of attributed. This makes it very hard to query the data and use it, rather we need to define all the "ways" that are connected ("nodes" are the key!), connect them, calculate the total length (using WGS84 ellipsoid), and use one attribute for each set of attributes of segments forming a single Road/Highway.


Please note that this code is a work-in-progress.

In [1]:
## Import Libraries and query OpenStreetMap API for Toll Facilities in California ##
##      Change California to a name of another state or country as desired        ##

import overpy
import pandas as pd

api = overpy.Overpass()
result = api.query("""area[name="California"][boundary=administrative]->.searchArea;
// gather results
(
  // query part for: “toll=yes”
  node["toll"="yes"]["highway"="motorway"](area.searchArea);
  way["toll"="yes"]["highway"="motorway"](area.searchArea);
//  relation["type"="route"]["route"="road"](area.searchArea);

//relation["toll"="yes"](area.searchArea);
);
// print results
out;
>;
out skel qt;""")

out_ways = []
for way in result.ways:
    out_ways.append(way)

out_relations = []
for relation in result.relations:
    out_relations.append(relation)

out_nodes = []
for node in result.nodes:
    out_nodes.append(node)

In [ ]:
## Check ##

for way in result.ways:
    print("ID: %s" % way.id)
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Toll: %s" % way.tags.get("toll", "n/a"))
    print("  Ref: %s" % way.tags.get("ref", "n/a"))
    print("  Bridge: %s" % way.tags.get("bridge", "no"))
    print("  County: %s" % way.tags.get("tiger:county", "n/a"))
    print("  Name_Base: %s" % way.tags.get("tiger:name_base", "n/a"))
    print("  Name_Base1: %s" % way.tags.get("tiger:name_base_1", "n/a"))
    print("  Name_Base2: %s" % way.tags.get("tiger:name_base_2", "n/a"))
    print("  Name_Base_Type: %s" % way.tags.get("tiger:name_type", "n/a"))
    print("  Name_Base_Type: %s" % way.tags.get("tiger:name_type", "n/a"))

In [2]:
## Extract ways from results to list, extract to csv ##

WaysList = []
for way in result.ways:
    Way_ID = way.id
    WaysList.append(Way_ID)
Ways_df = pd.DataFrame(WaysList)
Ways_df.to_csv("Ways-CA.csv", header=1)

### Nodes Extracting and Wrangling

In [3]:
## Ways & Nodes extraction ##

way_of_node_list = []
NodesList = []
for way in result.ways:
    way_of_node = way
    Nodes_in_way = way.nodes
    way_of_node_list.append(way_of_node)
    NodesList.append(Nodes_in_way)
Nodes_df_w_way = pd.DataFrame(way_of_node_list)
Nodes_df = pd.DataFrame(NodesList)
Nodes_df_w_way.to_csv("Nodes_w_Ways-CA.csv", header=1)

In [4]:
Nodes_df_w_way = Nodes_df_w_way.iloc[:, 0].map(lambda x: str(x)[15:])
Nodes_df_w_way = Nodes_df_w_way.apply(lambda x: pd.Series(x.split(' nodes=')))
Nodes_df_w_way = Nodes_df_w_way.iloc[:, 0]

In [5]:
## Create new dataframe; each row contains the Way id (Column 1), and all the nodes within that way (other columns) ##

Nodes_df_w_way = pd.concat((Nodes_df_w_way, Nodes_df), axis=1)
for i in range (1, len(Nodes_df_w_way.columns)):
    Nodes_df_w_way.iloc[:, i] = Nodes_df_w_way.iloc[:, i].map(lambda x: str(x)[16:-1])
Nodes_df_w_way

,0,0,1,2,3,4,5,6,7,8,...,48,49,50,51,52,53,54,55,56,57
0,4968908,32915539 lat=38.0317251 lon=-122.1167537,663024653 lat=38.0336536 lon=-122.1167138,32915540 lat=38.0341245 lon=-122.1167195,663024663 lat=38.0345490 lon=-122.1167539,663024676 lat=38.0350959 lon=-122.1168368,663024682 lat=38.0357672 lon=-122.1169916,3696071393 lat=38.0361732 lon=-122.1171189,663024692 lat=38.0364470 lon=-122.1172239,32915541 lat=38.0367557 lon=-122.1173503,...,,,,,,,,,,
1,8698972,32915482 lat=38.0485796 lon=-122.1261793,4375922677 lat=38.0487189 lon=-122.1263516,4375922678 lat=38.0491424 lon=-122.1266909,663028746 lat=38.0498942 lon=-122.1273680,663028727 lat=38.0503430 lon=-122.1277643,663028734 lat=38.0505542 lon=-122.1279736,663028741 lat=38.0507330 lon=-122.1281933,4375922685 lat=38.0508681 lon=-122.1283975,663028742 lat=38.0510300 lon=-122.1286857,...,,,,,,,,,,
2,8703049,32915482 lat=38.0485796 lon=-122.1261793,3982309558 lat=38.0487625 lon=-122.1262621,663025823 lat=38.0499670 lon=-122.1271957,4375922683 lat=38.0506413 lon=-122.1277006,,,,,,...,,,,,,,,,,
3,8929962,295568782 lat=37.4980075 lon=-122.1301033,523830025 lat=37.4975081 lon=-122.1306662,523830129 lat=37.4970314 lon=-122.1312069,271289919 lat=37.4966058 lon=-122.1316658,523866177 lat=37.4961904 lon=-122.1320950,523866181 lat=37.4956048 lon=-122.1326700,523865814 lat=37.4950736 lon=-122.1331936,523865600 lat=37.4945220 lon=-122.1337773,257556153 lat=37.4940181 lon=-122.1343266,...,,,,,,,,,,
4,11192273,667726763 lat=37.8266629 lon=-122.2995887,99582500 lat=37.8263292 lon=-122.3014156,99582504 lat=37.8260559 lon=-122.3029606,,,,,,,...,,,,,,,,,,
5,11198593,99582504 lat=37.8260559 lon=-122.3029606,99582505 lat=37.8259287 lon=-122.3036848,367287966 lat=37.8257148 lon=-122.3049213,667724881 lat=37.8256745 lon=-122.3052431,99582506 lat=37.8256533 lon=-122.3055891,367287974 lat=37.8256576 lon=-122.3059191,99582507 lat=37.8256787 lon=-122.3062624,99582510 lat=37.8258186 lon=-122.3074721,99582512 lat=37.8258419 lon=-122.3078449,...,,,,,,,,,,
6,11198595,30363887 lat=37.8254299 lon=-122.3102678,430047798 lat=37.8253757 lon=-122.3105682,430047799 lat=37.8253350 lon=-122.3108429,430047800 lat=37.8253079 lon=-122.3111004,430047801 lat=37.8252943 lon=-122.3113664,430047802 lat=37.8252943 lon=-122.3116411,31652107 lat=37.8253011 lon=-122.3119672,31652108 lat=37.8253146 lon=-122.3124393,31652109 lat=37.8253146 lon=-122.3128084,...,,,,,,,,,,
7,12181069,249059861 lat=37.9325533 lon=-122.4062800,666738703 lat=37.9325611 lon=-122.4065786,666738704 lat=37.9325886 lon=-122.4071687,249059862 lat=37.9326267 lon=-122.4077212,2245787553 lat=37.9330970 lon=-122.4142876,2245787554 lat=37.9335296 lon=-122.4208681,977948700 lat=37.9343872 lon=-122.4339956,110416699 lat=37.9353462 lon=-122.4480691,263611043 lat=37.9354025 lon=-122.4487963,...,,,,,,,,,,
8,13337301,122916785 lat=33.7380781 lon=-117.7176896,3235659602 lat=33.7376995 lon=-117.7176897,3235659603 lat=33.7373958 lon=-117.7177087,122916782 lat=33.7372361 lon=-117.7177225,,,,,,...,,,,,,,,,,
9,13353015,123004305 lat=33.6719156 lon=-117.7533111,123004299 lat=33.6729548 lon=-117.7533612,123004294 lat=33.6738453 lon=-117.7532840,331375704 lat=33.6743405 lon=-117.7532048,,,,,,...,,,,,,,,,,


In [6]:
##                            Transpose and set 1st row as header                               ## 
##  Now we have for each column the column name as the Way id, the column values are the nodes  ##

Nodes_df_w_way_T = Nodes_df_w_way.T
new_header = Nodes_df_w_way_T.iloc[0] # Grab the first row for the header
Nodes_df_w_way_T = Nodes_df_w_way_T[1:] # Take the data less the header row
Nodes_df_w_way_T.columns = new_header # Set the header row as the df header
Nodes_df_w_way_T

,4968908,8698972,8703049,8929962,11192273,11198593,11198595,12181069,13337301,13353015,...,482532988,482532989,482532990,484661658,484661659,492214227,492214228,517934789,537838948,562568903
0,32915539 lat=38.0317251 lon=-122.1167537,32915482 lat=38.0485796 lon=-122.1261793,32915482 lat=38.0485796 lon=-122.1261793,295568782 lat=37.4980075 lon=-122.1301033,667726763 lat=37.8266629 lon=-122.2995887,99582504 lat=37.8260559 lon=-122.3029606,30363887 lat=37.8254299 lon=-122.3102678,249059861 lat=37.9325533 lon=-122.4062800,122916785 lat=33.7380781 lon=-117.7176896,123004305 lat=33.6719156 lon=-117.7533111,...,1832733489 lat=33.8834665 lon=-117.6266695,4291817383 lat=33.8837629 lon=-117.6280250,988458491 lat=33.8800025 lon=-117.6546425,4774157689 lat=37.5355485 lon=-122.0750344,523230238 lat=37.5365042 lon=-122.0730103,4842640476 lat=38.0149719 lon=-121.7514856,4842640475 lat=38.0151480 lon=-121.7514803,2443808081 lat=37.8114175 lon=-122.3637551,3718987342 lat=37.8081054 lon=-122.4765197,667724547 lat=37.8246893 lon=-122.3143707
1,663024653 lat=38.0336536 lon=-122.1167138,4375922677 lat=38.0487189 lon=-122.1263516,3982309558 lat=38.0487625 lon=-122.1262621,523830025 lat=37.4975081 lon=-122.1306662,99582500 lat=37.8263292 lon=-122.3014156,99582505 lat=37.8259287 lon=-122.3036848,430047798 lat=37.8253757 lon=-122.3105682,666738703 lat=37.9325611 lon=-122.4065786,3235659602 lat=33.7376995 lon=-117.7176897,123004299 lat=33.6729548 lon=-117.7533612,...,4291817382 lat=33.8837437 lon=-117.6274809,4525220141 lat=33.8834093 lon=-117.6269715,1832710289 lat=33.8802638 lon=-117.6537795,343604586 lat=37.5355173 lon=-122.0751045,523230250 lat=37.5362047 lon=-122.0736025,524704018 lat=38.0150774 lon=-121.7514777,3968891052 lat=38.0155553 lon=-121.7514952,621522398 lat=37.8107987 lon=-122.3644016,5207344513 lat=37.8082105 lon=-122.4766288,667724548 lat=37.8245707 lon=-122.3149715
2,32915540 lat=38.0341245 lon=-122.1167195,4375922678 lat=38.0491424 lon=-122.1266909,663025823 lat=38.0499670 lon=-122.1271957,523830129 lat=37.4970314 lon=-122.1312069,99582504 lat=37.8260559 lon=-122.3029606,367287966 lat=37.8257148 lon=-122.3049213,430047799 lat=37.8253350 lon=-122.3108429,666738704 lat=37.9325886 lon=-122.4071687,3235659603 lat=33.7373958 lon=-117.7177087,123004294 lat=33.6738453 lon=-117.7532840,...,,,988458494 lat=33.8804080 lon=-117.6531953,4774157688 lat=37.5354903 lon=-122.0751655,53003034 lat=37.5358848 lon=-122.0742806,4842640475 lat=38.0151480 lon=-121.7514803,,,5207344723 lat=37.8083150 lon=-122.4767297,667724549 lat=37.8244393 lon=-122.3155696
3,663024663 lat=38.0345490 lon=-122.1167539,663028746 lat=38.0498942 lon=-122.1273680,4375922683 lat=38.0506413 lon=-122.1277006,271289919 lat=37.4966058 lon=-122.1316658,,667724881 lat=37.8256745 lon=-122.3052431,430047800 lat=37.8253079 lon=-122.3111004,249059862 lat=37.9326267 lon=-122.4077212,122916782 lat=33.7372361 lon=-117.7177225,331375704 lat=33.6743405 lon=-117.7532048,...,,,1832710290 lat=33.8806650 lon=-117.6521320,,4281421806 lat=37.5355997 lon=-122.0749197,,,,5207344724 lat=37.8084318 lon=-122.4768312,367288116 lat=37.8239833 lon=-122.3177003
4,663024676 lat=38.0350959 lon=-122.1168368,663028727 lat=38.0503430 lon=-122.1277643,,523866177 lat=37.4961904 lon=-122.1320950,,99582506 lat=37.8256533 lon=-122.3055891,430047801 lat=37.8252943 lon=-122.3113664,2245787553 lat=37.9330970 lon=-122.4142876,,,...,,,988458496 lat=33.8808538 lon=-117.6513641,,4774157689 lat=37.5355485 lon=-122.0750344,,,,5207344503 lat=37.8085387 lon=-122.4769154,667724550 lat=37.8239076 lon=-122.3180748
5,663024682 lat=38.0357672 lon=-122.1169916,663028734 lat=38.0505542 lon=-122.1279736,,523866181 lat=37.4956048 lon=-122.1326700,,367287974 lat=37.8256576 lon=-122.3059191,430047802 lat=37.8252943 lon=-122.3116411,2245787554 lat=37.9335296 lon=-122.4208681,,,...,,,988458498 lat=33.8812339 lon=-117.6498253,,,,,,5207344520 lat=37.8086457 lon=-122.4769928,667724551 lat=37.8237656 lon=-122.3186542
6,3696071393 lat=38.0361732

In [7]:
## Create Dummy DataFrames ##

import numpy as np

d = pd.DataFrame(np.zeros((len(Nodes_df_w_way_T), len(Nodes_df_w_way_T.columns))))
d2 = pd.DataFrame(np.zeros((len(Nodes_df_w_way_T), len(Nodes_df_w_way_T.columns))))
d3 = pd.DataFrame(np.zeros((len(Nodes_df_w_way_T), len(Nodes_df_w_way_T.columns))))

In [8]:
##  Split at substrings lon=, lat=  ##

foo1 = lambda x: pd.Series([i for i in reversed(x.split('lat='))])
for i in range (0, len(Nodes_df_w_way_T.columns)):
    d[i] = Nodes_df_w_way_T.iloc[:, i].apply(foo1)
foo2 = lambda x: pd.Series([i for i in reversed(x.split('lon='))])
for i in range (0, len(Nodes_df_w_way_T.columns)):
    d2[i] = Nodes_df_w_way_T.iloc[:, i].apply(foo2)
foo3 = lambda x: pd.Series([i for i in (x.split('lon='))]);
for i in range (0, len(Nodes_df_w_way_T.columns)):
    d3[i] = d.iloc[:, i].apply(foo3)

In [9]:
foo4 = lambda x: pd.Series([i for i in (x.split('lat='))]); 
for i in range (0, len(Nodes_df_w_way_T.columns)):
    d.iloc[:, i] = Nodes_df_w_way_T.iloc[:, i].apply(foo4)
d

,0,1,2,3,4,5,6,7,8,9,...,429,430,431,432,433,434,435,436,437,438
0,32915539,32915482,32915482,295568782,667726763,99582504,30363887,249059861,122916785,123004305,...,1832733489,4291817383,988458491,4774157689,523230238,4842640476,4842640475,2443808081,3718987342,667724547
1,663024653,4375922677,3982309558,523830025,99582500,99582505,430047798,666738703,3235659602,123004299,...,4291817382,4525220141,1832710289,343604586,523230250,524704018,3968891052,621522398,5207344513,667724548
2,32915540,4375922678,663025823,523830129,99582504,367287966,430047799,666738704,3235659603,123004294,...,,,988458494,4774157688,53003034,4842640475,,,5207344723,667724549
3,663024663,663028746,4375922683,271289919,,667724881,430047800,249059862,122916782,331375704,...,,,1832710290,,4281421806,,,,5207344724,367288116
4,663024676,663028727,,523866177,,99582506,430047801,2245787553,,,...,,,988458496,,4774157689,,,,5207344503,667724550
5,663024682,663028734,,523866181,,367287974,430047802,2245787554,,,...,,,988458498,,,,,,5207344520,667724551
6,3696071393,663028741,,523865814,,99582507,31652107,977948700,,,...,,,988458501,,,,,,5207344514,99582625
7,663024692,4375922685,,523865600,,99582510,31652108,110416699,,,...,,,988458305,,,,,,5207344498,667491632
8,32915541,663028742,,257556153,,99582512,31652109,263611043,,,...,,,4291817379,,,,,,5207344721,
9,663024704,62213749,,3983486567,,99582513,430047854,666738743,,,...,,,4291817380,,,,,,5207344515,


In [10]:
## Rename columns ##

d.columns = Nodes_df_w_way_T.columns
d2.columns = Nodes_df_w_way_T.columns
d3.columns = Nodes_df_w_way_T.columns

In [11]:
nodes_df = d.copy()
lat_df = d3.copy()
lon_df = d2.copy()
len_df = d.copy()

In [12]:
## Add suffix to columns ##

nodes_df = nodes_df.add_suffix(' Node_ID')
lat_df = lat_df.add_suffix(' lat')
lon_df = lon_df.add_suffix(' long')
len_df = len_df.add_suffix(' Length')

In [13]:
len_df

,4968908 Length,8698972 Length,8703049 Length,8929962 Length,11192273 Length,11198593 Length,11198595 Length,12181069 Length,13337301 Length,13353015 Length,...,482532988 Length,482532989 Length,482532990 Length,484661658 Length,484661659 Length,492214227 Length,492214228 Length,517934789 Length,537838948 Length,562568903 Length
0,32915539,32915482,32915482,295568782,667726763,99582504,30363887,249059861,122916785,123004305,...,1832733489,4291817383,988458491,4774157689,523230238,4842640476,4842640475,2443808081,3718987342,667724547
1,663024653,4375922677,3982309558,523830025,99582500,99582505,430047798,666738703,3235659602,123004299,...,4291817382,4525220141,1832710289,343604586,523230250,524704018,3968891052,621522398,5207344513,667724548
2,32915540,4375922678,663025823,523830129,99582504,367287966,430047799,666738704,3235659603,123004294,...,,,988458494,4774157688,53003034,4842640475,,,5207344723,667724549
3,663024663,663028746,4375922683,271289919,,667724881,430047800,249059862,122916782,331375704,...,,,1832710290,,4281421806,,,,5207344724,367288116
4,663024676,663028727,,523866177,,99582506,430047801,2245787553,,,...,,,988458496,,4774157689,,,,5207344503,667724550
5,663024682,663028734,,523866181,,367287974,430047802,2245787554,,,...,,,988458498,,,,,,5207344520,667724551
6,3696071393,663028741,,523865814,,99582507,31652107,977948700,,,...,,,988458501,,,,,,5207344514,99582625
7,663024692,4375922685,,523865600,,99582510,31652108,110416699,,,...,,,988458305,,,,,,5207344498,667491632
8,32915541,663028742,,257556153,,99582512,31652109,263611043,,,...,,,4291817379,,,,,,5207344721,
9,663024704,62213749,,3983486567,,99582513,430047854,666738743,,,...,,,4291817380,,,,,,5207344515,


In [14]:
## Concatenate the three dataframes ##

df = pd.concat([nodes_df, lat_df, lon_df, len_df], axis=1)
df

,4968908 Node_ID,8698972 Node_ID,8703049 Node_ID,8929962 Node_ID,11192273 Node_ID,11198593 Node_ID,11198595 Node_ID,12181069 Node_ID,13337301 Node_ID,13353015 Node_ID,...,482532988 Length,482532989 Length,482532990 Length,484661658 Length,484661659 Length,492214227 Length,492214228 Length,517934789 Length,537838948 Length,562568903 Length
0,32915539,32915482,32915482,295568782,667726763,99582504,30363887,249059861,122916785,123004305,...,1832733489,4291817383,988458491,4774157689,523230238,4842640476,4842640475,2443808081,3718987342,667724547
1,663024653,4375922677,3982309558,523830025,99582500,99582505,430047798,666738703,3235659602,123004299,...,4291817382,4525220141,1832710289,343604586,523230250,524704018,3968891052,621522398,5207344513,667724548
2,32915540,4375922678,663025823,523830129,99582504,367287966,430047799,666738704,3235659603,123004294,...,,,988458494,4774157688,53003034,4842640475,,,5207344723,667724549
3,663024663,663028746,4375922683,271289919,,667724881,430047800,249059862,122916782,331375704,...,,,1832710290,,4281421806,,,,5207344724,367288116
4,663024676,663028727,,523866177,,99582506,430047801,2245787553,,,...,,,988458496,,4774157689,,,,5207344503,667724550
5,663024682,663028734,,523866181,,367287974,430047802,2245787554,,,...,,,988458498,,,,,,5207344520,667724551
6,3696071393,663028741,,523865814,,99582507,31652107,977948700,,,...,,,988458501,,,,,,5207344514,99582625
7,663024692,4375922685,,523865600,,99582510,31652108,110416699,,,...,,,988458305,,,,,,5207344498,667491632
8,32915541,663028742,,257556153,,99582512,31652109,263611043,,,...,,,4291817379,,,,,,5207344721,
9,663024704,62213749,,3983486567,,99582513,430047854,666738743,,,...,,,4291817380,,,,,,5207344515,


In [15]:
## To CSV ##
df.to_csv('df_for_distances.csv', header =1)

In [16]:
## Parameter Length_cols = number of columns of "Length"
Length_cols = [col for col in df.columns if 'Length' in col]
len(Length_cols)

439

In [17]:
## Set default value ##

df = df.apply(lambda x: x.str.strip()).replace('', np.nan)
df.fillna(value=0.00001, inplace=True)
for i in range (0, len(Length_cols)):
    df['{}'.format(Length_cols[i])][df['{}'.format(Length_cols[i])] != 0.00001] = 0.0

In [18]:
for col in Length_cols:
    df.loc[:, col] = df.loc[:, col].astype(float)

In [19]:
## Check ##
len(df.columns)

1756

In [20]:
##  Backup  ##
df_backup = df.copy()
#df.reindex_axis(sorted(df.columns), axis=1)

In [21]:
##  Calculate distances between every two consecutive nodes, then sum distances within each way  ##
import geopy as gp
import math
from geopy.distance import vincenty

df = df_backup.copy()

Col_counter1 = len(Length_cols)
Col_counter2 = len(Length_cols)*2
for i in range (0, len(Length_cols)):
    for x in range (0, (len(df.iloc[:, i])-1)):
        x_1 = x + 1
        if (df.iloc[x_1, Col_counter1] != 0.00001):
            
            lat1 = df.iloc[x, Col_counter1]
            long1 = df.iloc[x, Col_counter2]
            lat2 = df.iloc[x+1, Col_counter1]
            long2 = df.iloc[x+1, Col_counter2]

            distance = vincenty((lat1, long1), (lat2, long2)).miles
            # print(distance)
            # df['{}'.format(Length_cols[i])][x] = distance
            df.loc[x, ('{}'.format(Length_cols[i]))] = distance
        
    Col_counter1 = Col_counter1 + 1
    Col_counter2 = Col_counter2 + 1

In [22]:
##  Test Distance: check against -- https://gps-coordinates.org/distance-between-coordinates.php  ##

for col in df.columns:
    if '90536418 ' in col:
        print(col)
        df[['90536418 Node_ID',
'90536418 lat',
'90536418 long',
'90536418 Length'
]]
        
lat1 = 37.6302786
long1 = -122.1151873
lat2 = 37.6190152
long2 = -122.1486514

distance2 = vincenty((lat1, long1), (lat2, long2)).miles
distance2

90536418 Node_ID
90536418 lat
90536418 long
90536418 Length


1.9932178632522668

In [23]:
##  Ways lengths  ##

Lengths_List = []

for i in range(0, len(Length_cols)):
    l = len(df['{}'.format(Length_cols[i])][df['{}'.format(Length_cols[i])] != 0.00001])
    #print(l)
    df['{}'.format(Length_cols[i])][l] = df['{}'.format(Length_cols[i])][0:l-1].sum()
    Lengths_List.append(df['{}'.format(Length_cols[i])][l])

/Users/askar/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
##  Dataframe with lengths  ##

df[Length_cols].head(22)

,4968908 Length,8698972 Length,8703049 Length,8929962 Length,11192273 Length,11198593 Length,11198595 Length,12181069 Length,13337301 Length,13353015 Length,...,482532988 Length,482532989 Length,482532990 Length,484661658 Length,484661659 Length,492214227 Length,492214228 Length,517934789 Length,537838948 Length,562568903 Length
0,0.133027,0.013439,0.013399,0.046290,0.102556,0.040577,0.016853,0.016320,0.026094,0.071680,...,0.050403,0.065279,0.052778,0.004410,0.038527,0.007289,0.028103,0.055432,0.009390,0.033870
1,0.032480,0.034578,0.097438,0.044311,0.086596,0.069233,0.015288,0.032291,0.020960,0.061535,...,0.000000,0.000000,0.035023,0.003833,0.043285,0.004871,0.000000,0.000000,0.009079,0.033952
2,0.029338,0.063658,0.054048,0.038695,0.000000,0.017822,0.014210,0.030295,0.011035,0.034433,...,0.050403,0.065279,0.063639,0.000000,0.040231,0.000000,0.028103,0.055432,0.009784,0.120731
3,0.037990,0.037753,0.000000,0.037106,0.189152,0.018984,0.014582,0.360162,0.000000,0.000000,...,0.000010,0.000010,0.046021,0.008243,0.007221,0.012160,0.000010,0.000010,0.008694,0.021142
4,0.047064,0.018506,0.164885,0.051276,0.000010,0.018055,0.015028,0.360702,0.058089,0.167648,...,0.000010,0.000010,0.092256,0.000010,0.000000,0.000010,0.000010,0.000010,0.008508,0.033176
5,0.028850,0.017194,0.000010,0.046581,0.000010,0.018837,0.017846,0.719531,0.000010,0.000010,...,0.000010,0.000010,0.150749,0.000010,0.129264,0.000010,0.000010,0.000010,0.008861,0.031135
6,0.019734,0.014521,0.000010,0.049757,0.000010,0.066877,0.025843,0.771602,0.000010,0.000010,...,0.000010,0.000010,0.139135,0.000010,0.000010,0.000010,0.000010,0.000010,0.008645,0.028841
7,0.022380,0.019281,0.000010,0.046029,0.000010,0.020457,0.020192,0.039912,0.000010,0.000010,...,0.000010,0.000010,0.138395,0.000010,0.000010,0.000010,0.000010,0.000010,0.009285,0.000000
8,0.039583,0.013209,0.000010,0.127519,0.000010,0.019817,0.017846,0.037151,0.000010,0.000010,...,0.000010,0.000010,0.096392,0.000010,0.000010,0.000010,0.000010,0.000010,0.008375,0.302847
9,0.044455,0.015784,0.000010,0.000000,0.000010,0.018394,0.016147,0.037407,0.000010,0.000010,...,0.000010,0.000010,0.116257,0.000010,0.000010,0.000010,0.000010,0.000010,0.007449,0.000010


In [25]:
##  Check number of columns  ##

print(len(Lengths_List))
print(len(Length_cols))

439
439


In [26]:
##  Dictionary with ways and final lengths  ##

Lengths_dict = [{Length_cols[i] : Lengths_List[i]} for i in range (0, len(Length_cols) -1)]
#print(Lengths_dict)

In [27]:
## To CSV ##

df.to_csv("df_w_distance.csv", header = 1)
df[Length_cols].to_csv("df_Length_cols_only.csv", header = 1)

In [31]:
#del Ways_w_Lengths

Ways_w_Lengths = pd.DataFrame(Length_cols)
Ways_w_Lengths['Length'] = pd.Series(Lengths_List)
Ways_w_Lengths.columns = ['Way', 'Length']
Ways_w_Lengths['Way'] = Ways_w_Lengths['Way'].map(lambda x: str(x)[:-7])
Ways_w_Lengths.head(8)


,Way,Length
0,4968908,1.298790
1,8698972,0.280449
2,8703049,0.164885
3,8929962,0.487565
4,11192273,0.189152
5,11198593,0.407063
6,11198595,0.540911
7,12181069,3.995194


## Identify Duplicate Ways

In [32]:
import numpy as np
Nodes_col = []
for i in range(0, len(df.columns)):
    if 'Node_ID' in df.columns[i]:
        Nodes_col.append(df.columns[i])
df_nocoord = df[Nodes_col]
df_nocoord

,4968908 Node_ID,8698972 Node_ID,8703049 Node_ID,8929962 Node_ID,11192273 Node_ID,11198593 Node_ID,11198595 Node_ID,12181069 Node_ID,13337301 Node_ID,13353015 Node_ID,...,482532988 Node_ID,482532989 Node_ID,482532990 Node_ID,484661658 Node_ID,484661659 Node_ID,492214227 Node_ID,492214228 Node_ID,517934789 Node_ID,537838948 Node_ID,562568903 Node_ID
0,32915539,32915482,32915482,295568782,667726763,99582504,30363887,249059861,122916785,123004305,...,1832733489,4291817383,988458491,4774157689,523230238,4842640476,4842640475,2443808081,3718987342,667724547
1,663024653,4375922677,3982309558,523830025,99582500,99582505,430047798,666738703,3235659602,123004299,...,4291817382,4525220141,1832710289,343604586,523230250,524704018,3968891052,621522398,5207344513,667724548
2,32915540,4375922678,663025823,523830129,99582504,367287966,430047799,666738704,3235659603,123004294,...,1e-05,1e-05,988458494,4774157688,53003034,4842640475,1e-05,1e-05,5207344723,667724549
3,663024663,663028746,4375922683,271289919,1e-05,667724881,430047800,249059862,122916782,331375704,...,1e-05,1e-05,1832710290,1e-05,4281421806,1e-05,1e-05,1e-05,5207344724,367288116
4,663024676,663028727,1e-05,523866177,1e-05,99582506,430047801,2245787553,1e-05,1e-05,...,1e-05,1e-05,988458496,1e-05,4774157689,1e-05,1e-05,1e-05,5207344503,667724550
5,663024682,663028734,1e-05,523866181,1e-05,367287974,430047802,2245787554,1e-05,1e-05,...,1e-05,1e-05,988458498,1e-05,1e-05,1e-05,1e-05,1e-05,5207344520,667724551
6,3696071393,663028741,1e-05,523865814,1e-05,99582507,31652107,977948700,1e-05,1e-05,...,1e-05,1e-05,988458501,1e-05,1e-05,1e-05,1e-05,1e-05,5207344514,99582625
7,663024692,4375922685,1e-05,523865600,1e-05,99582510,31652108,110416699,1e-05,1e-05,...,1e-05,1e-05,988458305,1e-05,1e-05,1e-05,1e-05,1e-05,5207344498,667491632
8,32915541,663028742,1e-05,257556153,1e-05,99582512,31652109,263611043,1e-05,1e-05,...,1e-05,1e-05,4291817379,1e-05,1e-05,1e-05,1e-05,1e-05,5207344721,1e-05
9,663024704,62213749,1e-05,3983486567,1e-05,99582513,430047854,666738743,1e-05,1e-05,...,1e-05,1e-05,4291817380,1e-05,1e-05,1e-05,1e-05,1e-05,5207344515,1e-05


In [33]:
##  Check  ##

print(len(df_nocoord.iloc[:]))
print(len(df_nocoord.columns))

58
439


In [34]:
##  Adjust Settings  ##

pd.set_option('display.max_rows', 100) and pd.set_option('display.max_rows', 100)

In [35]:
all_data = df_nocoord.values.tolist()
import itertools
merged = list(itertools.chain(*all_data))
print(len(merged))
all_data = pd.DataFrame(merged)
print(len(all_data))
cols_list = list(df_nocoord.columns)*len(df_nocoord.iloc[:, 0])
print(len(cols_list))

25462
25462
25462


In [36]:
##  Add Column 'Way'  ##

all_data['Way'] = pd.Series(cols_list)

In [37]:
## Test ##

all_data.iloc[1024, :]

0                 1e-05
Way    60178111 Node_ID
Name: 1024, dtype: object

In [38]:
##  Add Column for dups  ##

all_data['Dup_Value'] = np.zeros(len(all_data.iloc[:]))
All_df = all_data.copy()
All_df.columns = ['Node', 'Way', 'Dup_Value']
All_df['Way'] = All_df['Way'].map(lambda x: str(x)[:-8])
print(len(All_df))
All_df = All_df[All_df['Node'] != 1e-05]
print(len(All_df))

25462
3733


In [39]:
All_df.head(5)

,Node,Way,Dup_Value
0,32915539,4968908,0.0
1,32915482,8698972,0.0
2,32915482,8703049,0.0
3,295568782,8929962,0.0
4,667726763,11192273,0.0


In [40]:
##  Create list with all the dups  ##

l = list(All_df['Node'])
w = list(All_df['Way'])
dups = [[x, l.count(x)] for x in set(l)]
dups_df = pd.DataFrame(dups)
dups_df.columns = ['Node', 'Occurance']

In [41]:
##  Identify sets of two, three, four, ... eight ways that intersect at a certain point  ##

two_way_connectors = pd.DataFrame(dups)
two_way_connectors = dups_df[dups_df.iloc[:, 1] == 2]
print(len(two_way_connectors))

three_way_connectors = pd.DataFrame(dups)
three_way_connectors = dups_df[dups_df.iloc[:, 1] == 3]
print(len(three_way_connectors))

four_way_connectors = pd.DataFrame(dups)
four_way_connectors = dups_df[dups_df.iloc[:, 1] == 4]
print(len(four_way_connectors))

five_way_connectors = pd.DataFrame(dups)
five_way_connectors = dups_df[dups_df.iloc[:, 1] == 5]
print(len(five_way_connectors))

six_way_connectors = pd.DataFrame(dups)
six_way_connectors = dups_df[dups_df.iloc[:, 1] == 6]
print(len(six_way_connectors))

svn_way_connectors = pd.DataFrame(dups)
svn_way_connectors = dups_df[dups_df.iloc[:, 1] == 7]
print(len(svn_way_connectors))

eight_way_connectors = pd.DataFrame(dups)
eight_way_connectors = dups_df[dups_df.iloc[:, 1] == 8]
print(len(eight_way_connectors))

nine_way_connectors = pd.DataFrame(dups)
nine_way_connectors = dups_df[dups_df.iloc[:, 1] == 9]
print(len(nine_way_connectors))

396
12
0
0
0
0
0
0


In [42]:
##  Extract API calls results  ##

My_ways = []
for way in result.ways:
    My_ways.append(["Way ID ", way.id])
    My_ways.append(["Highway ", way.tags.get("highway", "n/a")])
    My_ways.append(["Toll ", way.tags.get("toll", "n/a")])
    My_ways.append(["Reference ", way.tags.get("ref", "n/a")])
    My_ways.append(["Bridge ", way.tags.get("bridge", "n/a")])
    My_ways.append(["County ", way.tags.get("tiger:county", "n/a")])
    My_ways.append(["Name ", way.tags.get("name", "n/a")])
    My_ways.append(["Name2 ", way.tags.get("tiger:name_base", "n/a")])
    My_ways.append(["Name3 ", way.tags.get("tiger:name_base_1", "n/a")])
    My_ways.append(["Name4 ", way.tags.get("tiger:name_base_2", "n/a")])
    My_ways.append(["Name Type ", way.tags.get("tiger:name_type", "n/a")])

My_relations = []
for relation in result.ways:
    My_relations.append(["Relation ID ", relation.id])
    My_relations.append(["Highway ", relation.tags.get("highway", "n/a")])
    My_relations.append(["Toll ",relation.tags.get("toll", "n/a")])
    My_relations.append(["Reference ", relation.tags.get("ref", "n/a")])
    My_relations.append(["Bridge ", relation.tags.get("bridge", "n/a")])
    My_relations.append(["County ", relation.tags.get("tiger:county", "n/a")])
    My_relations.append(["Name ", relation.tags.get("name", "n/a")])
    My_relations.append(["Name2 ", relation.tags.get("tiger:name_base", "n/a")])
    My_relations.append(["Name3 ", relation.tags.get("tiger:name_base_1", "n/a")])
    My_relations.append(["Name4 ", relation.tags.get("tiger:name_base_2", "n/a")])
    My_relations.append(["Name Type ", relation.tags.get("tiger:name_type", "n/a")])

My_nodes = []
for node in result.nodes:
    My_nodes.append(["Node ID ", node.id])
    My_nodes.append(["Highnode ", node.tags.get("highnode", "n/a")])
    My_nodes.append(["Toll ", node.tags.get("toll", "n/a")])
    My_nodes.append(["Reference ", node.tags.get("ref", "n/a")])
    My_nodes.append(["Bridge ", node.tags.get("bridge", "n/a")])
    My_nodes.append(["County ", node.tags.get("tiger:county", "n/a")])
    My_nodes.append(["Name ", node.tags.get("name", "n/a")])
    My_nodes.append(["Name2 ", node.tags.get("tiger:name_base", "n/a")])
    My_nodes.append(["Name3 ", node.tags.get("tiger:name_base_1", "n/a")])
    My_nodes.append(["Name4 ", node.tags.get("tiger:name_base_2", "n/a")])
    My_nodes.append(["Name Type ", node.tags.get("tiger:name_type", "n/a")])

In [44]:
##  To dataframe  ##

My_ways = pd.DataFrame(My_ways)
My_ways.columns = ['Attribute', 'Value']


## Two Way Connectors

In [46]:
##  Each node and how many times it appeared  ##

two_way_connectors.head(6)

,Node,Occurance
0,33219729,2
18,57798203,2
26,331375709,2
36,1371548836,2
37,4385957083,2
48,3010007775,2


In [47]:
##  Sets of two ways IN ORDER (first 2 are first way, second 2 are second way etc.)  ##

TwoWays = All_df[All_df.iloc[:, 0].isin(list(two_way_connectors.iloc[:, 0]))]
TwoWays = TwoWays.sort_values(by='Node', ascending=True)
TwoWaysList = list(TwoWays.Way)
TwoWaysList_numeric = [int(x) for x in TwoWaysList]


In [48]:
##  Splitting Two Ways List into smaller lists  ##

section = 2
length = len(TwoWaysList_numeric)
split_into = int(length/section)
x = TwoWaysList_numeric
y = np.array_split(x, split_into)
Two = [[] for x in range(0,int(split_into))]
for i in range(0, int(split_into)):
    Two[i] = list(y[i])
    
print(len(Two))
Two[1]

396


[429970620, 90093997]

In [49]:
##  Get ID's of sets of Ways  ##

My_ways_of_Two = [[] for x in range(0, split_into)]
for i in range(0, split_into):
    My_ways_of_Two[i] = My_ways[My_ways.Value.isin(Two[i])]
My_ways_of_Two[1]

,Attribute,Value
2310,Way ID,90093997
4202,Way ID,429970620


In [50]:
##  Get Indices of sets of Ways  ##

My_ways_of_Two_indices = [[] for x in range(0, split_into)]
for i in range(0, split_into):
    My_ways_of_Two_indices[i] = list(My_ways_of_Two[i].index)
My_ways_of_Two_indices[0]

[2310, 2332]

In [51]:
My_ways_of_Two_indices = list(itertools.chain(*My_ways_of_Two_indices))

In [53]:
## All into one BIG dataframe  ##

Indicies_List_Str = [str(x) for x in My_ways_of_Two_indices]
dict_2 = {name: pd.DataFrame() for name in Indicies_List_Str}
tup_twos_indices = tuple(Indicies_List_Str)
tup_twos_indices[1]
for i in range (len(My_ways_of_Two_indices)):
    dict_2['{}'.format(tup_twos_indices[i])] = pd.DataFrame(np.zeros((11, 2)))
    dict_2['{}'.format(tup_twos_indices[i])].columns = ['Attribute', 'Value']
    dict_2['{}'.format(tup_twos_indices[i])] = My_ways.iloc[My_ways_of_Two_indices[i]: My_ways_of_Two_indices[i] + 11, :]
    #print(dict_2['{}'.format(tup_twos_indices[i])])
    
BIG_df = pd.DataFrame(np.zeros((0, 2)))
BIG_df.columns = ['Attribute', 'Value']

for i in range (len(My_ways_of_Two_indices)):
    m = Ways_w_Lengths[Ways_w_Lengths.Way == str(dict_2['{}'.format(tup_twos_indices[i])].iloc[0,1])]
    m = m.iloc[0, 1]
    #print(type(m))
    #print(m)
    dict_2['{}'.format(tup_twos_indices[i])]['Way Length'] = m
    BIG_df = BIG_df.append(dict_2['{}'.format(tup_twos_indices[i])])

/Users/askar/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [54]:
##  Split attributes every 11 values  (the 11 attributes of ways that we care about)  ##

BIG_df['Index'] = [x for x in range (0, len(BIG_df.iloc[:]))]
BIG_df = BIG_df[['Index', 'Attribute', 'Value', 'Way Length']]
for i in range (0, len(BIG_df['Index'])):
    if i % 11 != 0:
        BIG_df.iloc[i, 3] = 'NaN'
    else:
        pass;
BIG_df.head()

,Index,Attribute,Value,Way Length
2310,0,Way ID,90093997,0.02799
2311,1,Highway,motorway,NaN
2312,2,Toll,yes,NaN
2313,3,Reference,CA 91 Toll,NaN
2314,4,Bridge,yes,NaN


In [55]:
BIG_df.to_csv("BIG_df_of_twos.csv", header = 1)

## Three Ways Connectors

In [56]:
##  Identical to two ways connectors  ##

three_way_connectors

# THREE WAYS IN ORDER (first 3 are first way, second 3 are second way etc.)
ThreeWays = All_df[All_df.iloc[:, 0].isin(list(three_way_connectors.iloc[:, 0]))]
ThreeWays = ThreeWays.sort_values(by='Node', ascending=True)
ThreeWaysList = list(ThreeWays.Way)
ThreeWaysList_numeric = [int(x) for x in ThreeWaysList]
ThreeWaysList_numeric

# Splitting Three Ways List into smaller lists
section = 3
length = len(ThreeWaysList_numeric)
split_into = int(length/section)
x = ThreeWaysList_numeric
y = np.array_split(x, split_into)
Three = [[] for x in range(0,int(split_into))]
for i in range(0, int(split_into)):
    Three[i] = list(y[i])
    
print(len(Three))
Three[0]


My_ways_of_Three = [[] for x in range(0, split_into)]
for i in range(0, split_into):
    My_ways_of_Three[i] = My_ways[My_ways.Value.isin(Three[i])]
My_ways_of_Three[1]


My_ways_of_Three_indices = [[] for x in range(0, split_into)]
for i in range(0, split_into):
    My_ways_of_Three_indices[i] = list(My_ways_of_Three[i].index)
My_ways_of_Three_indices[0]


My_ways_of_Three_indices = list(itertools.chain(*My_ways_of_Three_indices))
My_ways_of_Three_indices


List_of_dfs_threes = [[] for x in range(0, len(My_ways_of_Three_indices))]

for i in range (0, len(My_ways_of_Three_indices)):
    List_of_dfs_threes[i] = My_ways.iloc[My_ways_of_Three_indices[i]: My_ways_of_Three_indices[i] + 11, :]
    
List_of_dfs_threes = pd.DataFrame(List_of_dfs_threes)
List_of_dfs_threes.to_csv('THREE_Way_Connectors.csv', header=0)

12


# Relations

In [58]:

api = overpy.Overpass()
result = api.query("""area[name="California"][boundary=administrative]->.searchArea;
// gather results
(
  // query part for: “toll=yes”
  node["toll"="yes"]["highway"="motorway"](area.searchArea);
  way["toll"="yes"]["highway"="motorway"](area.searchArea);
  relation["type"="route"]["route"="road"](area.searchArea);

//relation["toll"="yes"](area.searchArea);
);
// print results
out;
>;
out skel qt;""")

out_relations = []
for relation in result.relations:
    out_relations.append(relation)


In [ ]:
##  Take a look at relations id in the API call results ##

for rel in result.relations:
    print(rel.id)

In [ ]:
##  Print relation id, tags  ##

for rel in result.ways:
    print("ID: %s" % rel.id)
    print("way: %s" % rel.tags.get("member", "n/a"))

In [60]:
##  List of ways that belong to a certain Relation  ##

rel_id_list = []
rel_ways_list = []
for rel in result.relations:
    rel_id_list.append(rel.id)
    rel_ways_list.append(rel.members)

In [61]:
len(rel_id_list)

646

In [62]:
##  Check  ##

rel_ways_list[0]

[<overpy.RelationWay ref=263844651 role=None>,
 <overpy.RelationWay ref=10586717 role=None>,
 <overpy.RelationWay ref=30445036 role=None>,
 <overpy.RelationWay ref=30444748 role=None>,
 <overpy.RelationWay ref=30444756 role=None>,
 <overpy.RelationWay ref=179380720 role=None>,
 <overpy.RelationWay ref=32152072 role=None>,
 <overpy.RelationWay ref=68309383 role=None>,
 <overpy.RelationWay ref=68309382 role=None>,
 <overpy.RelationWay ref=32152076 role=None>,
 <overpy.RelationWay ref=30445591 role=None>,
 <overpy.RelationWay ref=133443205 role=None>,
 <overpy.RelationWay ref=30445802 role=None>,
 <overpy.RelationWay ref=30445801 role=None>,
 <overpy.RelationWay ref=30446202 role=None>,
 <overpy.RelationWay ref=30446199 role=None>,
 <overpy.RelationWay ref=119400796 role=None>,
 <overpy.RelationWay ref=184703217 role=None>,
 <overpy.RelationWay ref=184703225 role=None>,
 <overpy.RelationWay ref=30446378 role=None>,
 <overpy.RelationWay ref=30446379 role=None>,
 <overpy.RelationWay ref=327

In [63]:
rel_ways_df = pd.DataFrame(rel_ways_list)
rel_ways_df = rel_ways_df.T
rel_ways_df.columns = [rel_id_list]
rel_ways_df.head()

,69363,69364,70350,71162,74660,74885,76279,76331,86957,93835,...,7667083,7667084,7667809,7667810,7667811,7669414,7669415,7669416,7714228,7765540
0,<overpy.RelationWay ref=263844651 role=None>,<overpy.RelationWay ref=254299122 role=west>,<overpy.RelationWay ref=158245363 role=forward>,<overpy.RelationWay ref=26467753 role=forward>,<overpy.RelationWay ref=550284583 role=None>,<overpy.RelationWay ref=125016692 role=None>,<overpy.RelationWay ref=30959004 role=None>,<overpy.RelationWay ref=30956245 role=None>,<overpy.RelationWay ref=45311025 role=forward>,<overpy.RelationWay ref=23309738 role=forward>,...,<overpy.RelationWay ref=518471311 role=None>,<overpy.RelationWay ref=441958165 role=None>,<overpy.RelationWay ref=443125586 role=None>,<overpy.RelationWay ref=523424492 role=None>,<overpy.RelationWay ref=533901233 role=None>,<overpy.RelationWay ref=226199725 role=None>,<overpy.RelationWay ref=534096611 role=None>,<overpy.RelationWay ref=534096608 role=None>,<overpy.RelationWay ref=538860612 role=backward>,<overpy.RelationWay ref=76028558 role=north>
1,<overpy.RelationWay ref=10586717 role=None>,<overpy.RelationWay ref=510583844 role=west>,<overpy.RelationWay ref=31559312 role=forward>,<overpy.RelationWay ref=117809223 role=forward>,<overpy.RelationWay ref=157782136 role=None>,<overpy.RelationWay ref=27422569 role=None>,<overpy.RelationWay ref=30959003 role=None>,<overpy.RelationWay ref=30956242 role=None>,<overpy.RelationWay ref=45311026 role=forward>,<overpy.RelationWay ref=51866293 role=forward>,...,<overpy.RelationWay ref=518471316 role=None>,<overpy.RelationWay ref=518471307 role=None>,None,None,<overpy.RelationWay ref=523424489 role=None>,<overpy.RelationWay ref=534096615 role=None>,<overpy.RelationWay ref=534096610 role=None>,<overpy.RelationWay ref=304919277 role=None>,<overpy.RelationWay ref=538860610 role=backward>,<overpy.RelationWay ref=76028556 role=north>
2,<overpy.RelationWay ref=30445036 role=None>,<overpy.RelationWay ref=31129732 role=west>,<overpy.RelationWay ref=31559315 role=forward>,<overpy.RelationWay ref=32867942 role=forward>,<overpy.RelationWay ref=123867439 role=forward>,<overpy.RelationWay ref=27422568 role=None>,<overpy.RelationWay ref=27807303 role=None>,<overpy.RelationWay ref=30956241 role=None>,<overpy.RelationWay ref=45311020 role=forward>,<overpy.RelationWay ref=123867346 role=forward>,...,<overpy.RelationWay ref=518471325 role=None>,<overpy.RelationWay ref=523421134 role=None>,None,None,None,<overpy.RelationWay ref=37247147 role=None>,<overpy.RelationWay ref=299536543 role=None>,None,<overpy.RelationWay ref=538860611 role=backward>,<overpy.RelationWay ref=10729676 role=north>
3,<overpy.RelationWay ref=30444748 role=None>,<overpy.RelationWay ref=202485364 role=west>,<overpy.RelationWay ref=31559316 role=forward>,<overpy.RelationWay ref=32867941 role=forward>,<overpy.RelationWay ref=552152112 role=forward>,<overpy.RelationWay ref=396832156 role=None>,<overpy.RelationWay ref=27807305 role=None>,<overpy.RelationWay ref=412858881 role=None>,<overpy.RelationWay ref=45311012 role=forward>,<overpy.RelationWay ref=123867345 role=forward>,...,<overpy.RelationWay ref=518471319 role=None>,<overpy.RelationWay ref=439964351 role=None>,None,None,None,None,None,None,<overpy.RelationWay ref=538860608 role=backward>,<overpy.RelationWay ref=76028563 role=north>
4,<overpy.RelationWay ref=30444756 role=None>,<overpy.RelationWay ref=120813417 role=west>,<overpy.RelationWay ref=498017763 role=forward>,<overpy.RelationWay ref=25287752 role=forward>,<overpy.RelationWay ref=29059985 role=forward>,<overpy.RelationWay ref=396832155 role=None>,<overpy.RelationWay ref=28357188 role=None>,<overpy.RelationWay ref=412858884 role=None>,<overpy.RelationWay ref=30930111 role=forward>,<overpy.RelationWay ref=46647901 role=forward>,...,<overpy.RelationWay ref=439436849 role=None>,<overpy.RelationWay ref=439436680 role=None>,None,None,None,None,None,None,<overpy.RelationWay ref=538860609 role=backward>,<overpy.RelationWay ref=27054948 

In [64]:
##  Clean dataframe: delete "<overpy.RelationWay ref=" from each cell  ##

forward_list = []
backward_list = []
four_char_list = []
for i in range(0, len(rel_ways_df.columns)):
    if "=forward" in str(rel_ways_df.iloc[0, i]): 
        forward_list.append(i)
    elif "=backward" in str(rel_ways_df.iloc[0, i]): 
        backward_list.append(i)
    else:
        four_char_list.append(i)

In [65]:
for i in range (0, len(rel_ways_df.columns)):
    if i in four_char_list:
        rel_ways_df.iloc[:, i] = rel_ways_df.iloc[:, i].map(lambda x: str(x)[24:-11])
    elif i in forward_list:
        rel_ways_df.iloc[:, i] = rel_ways_df.iloc[:, i].map(lambda x: str(x)[24:-14])
    elif i in backward_list:
        rel_ways_df.iloc[:, i] = rel_ways_df.iloc[:, i].map(lambda x: str(x)[24:-15])
rel_ways_df

,69363,69364,70350,71162,74660,74885,76279,76331,86957,93835,...,7667083,7667084,7667809,7667810,7667811,7669414,7669415,7669416,7714228,7765540
0,263844651,254299122,158245363,26467753,550284583,125016692,30959004,30956245,45311025,23309738,...,518471311,441958165,443125586,523424492,533901233,226199725,534096611,534096608,538860612,76028558
1,10586717,510583844,31559312,117809223,157782136,27422569,30959003,30956242,45311026,51866293,...,518471316,518471307,,,523424489,534096615,534096610,304919277,538860610,76028556
2,30445036,31129732,31559315,32867942,123867439 ro,27422568,27807303,30956241,45311020,123867346,...,518471325,523421134,,,,37247147,299536543,,538860611,10729676
3,30444748,202485364,31559316,32867941,552152112 ro,396832156,27807305,412858881,45311012,123867345,...,518471319,439964351,,,,,,,538860608,76028563
4,30444756,120813417,498017763,25287752,29059985 ro,396832155,28357188,412858884,30930111,46647901,...,439436849,439436680,,,,,,,538860609,27054948
5,179380720,8921938,28359106,10318894,29059984 ro,206884278,28357187,10488023,30930137,46647908,...,,,,,,,,,538860607,479190745
6,32152072,90077241,28359104,184029841,394222116 ro,526521287,28357178,412858882,30930094,46647907,...,,,,,,,,,179348668,479190748
7,68309383,394443191,28359068,164755807,394222114 ro,417273049,28357177,412858883,30930087,123867351,...,,,,,,,,,538860606,10729680
8,68309382,23874736,28359059,164755815,158615262 ro,417202176,157750991,10483095,30930076,46647906,...,,,,,,,,,10423054,76028602
9,32152076,11415208,28359016,7707845,395293863 ro,417202188,28357294,30956239,30930075,123867352,...,,,,,,,,,10423601,10734397


In [66]:
str(rel_ways_df.iloc[0, 1])

'254299122'